In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_pickle('train_df.pickle')

In [3]:
train_df_t = (train_df[["text", "label"]])
mod_data = train_df_t.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.15 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1835/12231 as positives and unlabeling the rest


In [4]:
mod_data['class_test'] = 0
mod_data.iloc[pos_sample,-1] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 0    33730
1     1835
Name: class_test, dtype: int64


In [5]:
mod_data

,text,label,class_test
29196,he should have fucked her in the ass,1,0
64149,bitches jus b so mad at the wrong ppl i dont g...,1,1
1107,Nor I you youre putrid Youre another white kno...,1,1
23403,Go back to your country bitch,1,0
9262,@JohnCornyn @Debbie22210 The only humane thing...,1,0
...,...,...,...
61275,@purrfect1509 White people who hate white peop...,0,0
35406,I consented to the dick not the sperm,0,0
42863,STRAIGHT PEOPLE ARE CRAZY,0,0
39148,Antifa member firebombs ICE facility and is ar...,0,0


In [6]:
x_data = mod_data['text'].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [7]:
x_data

array(['he should have fucked her in the ass',
       'bitches jus b so mad at the wrong ppl i dont get it ',
       'Nor I you youre putrid Youre another white knob thinking youre virtuous while you shame people from different cultures Youre in luck Reddit loves that builds up the ol ego  if theres one thing Reddits taught me its the intolerance of the supposed tolerant Youre not new youre commonplace Dealt with racist shits like you since birth Im all gd',
       ..., 'STRAIGHT PEOPLE ARE CRAZY',
       'Antifa member firebombs ICE facility and is armed with an AR15 Wrote a manifesto using Cowfarts Cortez idea that detention facilities are CONCENTRATION CAMPS Where is the Left on condemning him CUE CRICKETS',
       'Now there is another wackadoo called Shelle that she has shifted her allegiance to While not nice at least you can through some churchy sounding words point your mother to repentance for putting some whackadoodle over the actual leadership of the church'],
      dtype=ob

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [9]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [10]:
# bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [11]:
# bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [12]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.2)(net)
net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)

model = tf.keras.Model(text_input, net)

In [13]:
opt = Adam(learning_rate = 3e-5)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
# find the indices of the positive/labeled elements

positives = np.where(y_labeled == 1.)[0] 
print(positives)

[    1     2    14 ... 12206 12218 12219]


In [15]:
# hold_out_size = the *number* of positives/labeled samples 
# that we will use later to estimate P(s=1|y=1)
hold_out_ratio = 0.25
hold_out_size = int(np.ceil(len(positives) * hold_out_ratio))
np.random.shuffle(positives)

hold_out = positives[:hold_out_size] 
X_hold_out = x_data[hold_out]
X_hold_out[:5]

array(['So wise Go take a dick in your ass Fucking pedophile lover',
       'WITH NO FEMINISM OR LGBTHSGHDJDGSVXHDBULLSHIT THIS IS GOING TO BE A SUCCESS',
       'beta pussy you say Hmmm lets rearrange those letters Beat Pussy sounds fun but also violent Bate Pussy sounds like a sting operation Be at pussy sounds like a date  or my personal favorite I Bet a pussy like you only talks shit online',
       'Where did all of mankind invade gayboy oure just another of the gay assed demoncrapers who hate your own race arent you turdboy Now dont bother me any further turdboy Im not a gay assed demoncraper Im sure you have a goyfriendhusband who is interested in what the hell your gay ass had to say',
       'Ofc you want et fucked Chad you are a dumb slut Nothing but some holes'],
      dtype=object)

In [16]:
# the actual positive *elements* that we will keep aside
 
# remove the held out elements from X and y
X = np.delete(x_data, hold_out,0) 
y = np.delete(y_labeled, hold_out)

In [17]:
y[:5]

array([0, 1, 1, 0, 0], dtype=int64)

In [18]:
X_train,X_val, y_train,y_val = train_test_split(X,y,test_size= 0.2, stratify=y)
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 4,restore_best_weights =True)
print(np.asarray(X_train)[:5])
model.fit(np.asarray(list(X_train)).astype('str'),np.array(list(y_train),dtype = np.float),
          validation_data =( np.array(list(X_val)),np.asarray(list(y_val), dtype = np.float) ) ,epochs = 2, batch_size =  32,callbacks = [early_stopping])
#         # We then use the estimator for prediction of the positive held-out set 
# in order to estimate P(s=1|y=1)

['It pretty much is something like the black man recorded the mean white man saying he needs to leave the building that the white man lives in or at least call whoever invited the black man to the building to come show that he is allowed to be in the building what a mean white man The black man told the white man that hes gonna be on tv now'
 'Uhlbelk i agree but it wouldnt alter what i watch on youtube and TV i can empathise with the LGBT community but when i ask questions i dont get answers'
 'Real Americans indians  go back to Europe  White European colonialists  ah'
 'Lots of love from the Slavs we love you man'
 'Real sluts are so horny they open their legs and let their whore pussy be fucked by any cock no matter who it is attached to RT if youre a real slut like me and Natalie Comment to tell me how you want to use my fucktoy cumslut holes URL']
Epoch 1/2


C:\Users\Dhwanit\anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.
C:\Users\Dhwanit\anaconda3\envs\py3.7\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


878/878 [==============================] - 360s 403ms/step - loss: 0.1638 - accuracy: 0.9601 - val_loss: 0.1545 - val_accuracy: 0.9608
Epoch 2/2
878/878 [==============================] - 380s 433ms/step - loss: 0.1467 - accuracy: 0.9607 - val_loss: 0.1606 - val_accuracy: 0.9608


In [19]:
hold_out_predictions = model.predict(list(X_hold_out))
hold_out_predictions

array([[0.07808495],
       [0.00132191],
       [0.05685413],
       [0.08507171],
       [0.1174004 ],
       [0.02930635],
       [0.00889529],
       [0.08850829],
       [0.01437239],
       [0.07607862],
       [0.04724938],
       [0.04256497],
       [0.07636018],
       [0.05190324],
       [0.11131077],
       [0.09364726],
       [0.01391776],
       [0.12482388],
       [0.08137543],
       [0.06416839],
       [0.16097717],
       [0.03865369],
       [0.19110367],
       [0.03574643],
       [0.02288342],
       [0.10672419],
       [0.10937218],
       [0.17214721],
       [0.0346283 ],
       [0.03085391],
       [0.02612351],
       [0.00274899],
       [0.07018656],
       [0.14685676],
       [0.03645108],
       [0.08759806],
       [0.15316218],
       [0.15673263],
       [0.0935421 ],
       [0.18329747],
       [0.01240223],
       [0.09006228],
       [0.19047038],
       [0.06330332],
       [0.13662097],
       [0.06052304],
       [0.07089081],
       [0.011

In [20]:
#take the probability that it is 1
hold_out_predictions = hold_out_predictions[:,0]
# save the mean probability 
c = np.mean(hold_out_predictions)


def predict_PU_prob(X, estimator, prob_s1y1):
    predicted_s = estimator.predict_proba(X)
    predicted_s = predicted_s[:,1]
    return predicted_s / prob_s1y1

In [21]:
c

0.0910217

In [22]:
mod_data

,text,label,class_test
29196,he should have fucked her in the ass,1,0
64149,bitches jus b so mad at the wrong ppl i dont g...,1,1
1107,Nor I you youre putrid Youre another white kno...,1,1
23403,Go back to your country bitch,1,0
9262,@JohnCornyn @Debbie22210 The only humane thing...,1,0
...,...,...,...
61275,@purrfect1509 White people who hate white peop...,0,0
35406,I consented to the dick not the sperm,0,0
42863,STRAIGHT PEOPLE ARE CRAZY,0,0
39148,Antifa member firebombs ICE facility and is ar...,0,0


In [23]:
test_ds = pd.read_pickle('test_df.pickle')

In [24]:
y_pred =  model.predict(list(test_ds['text'].values))
y_pred

array([[0.03101655],
       [0.07421876],
       [0.12672599],
       ...,
       [0.00156461],
       [0.16288432],
       [0.00198944]], dtype=float32)

In [25]:
y_pred = y_pred/c

y_pred2 = [1 if i[0]>0.5 else 0 for i in y_pred]

In [26]:
from sklearn.metrics import classification_report

print(classification_report(test_ds['label'].values, y_pred2))

              precision    recall  f1-score   support

           0       0.75      0.71      0.73      2000
           1       0.73      0.77      0.75      2000

    accuracy                           0.74      4000
   macro avg       0.74      0.74      0.74      4000
weighted avg       0.74      0.74      0.74      4000



In [27]:
import os
 
pid = os.getpid()
#!kill -9 $pid